# Week 3 assignment

## PART 1: Getting the Data

Lets first import the librarys and data set we gonna use here

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]



Now the fun begins. For each column of the target frame I create a list.
Next, I loop through all the values of the data frame. Whenever there is a Not assigned in there, it is skipped. else I split the data.  

The first 3 cars are the Postal
The Borough is everything before the "(" (if there is a "(" ... else its just everything past the postal  
The Neighbours are everything after the "("... if there is no "(", there are no neighbours.   
The Neighbours are split by "/" which I replace to be split by ","  

foreach value I append the data to lists from which I construct the data frame. Thats it. 

In [2]:
postals = []
boroughs = []
neighbours = []

for value in df.values:
    for x in value:
        if not ("Not assigned" in x):
            #print(x)
            postal = x[0:3] # alwys like this
            
            # now the tricky part --- split by "("
            if "(" in x:
                borough = x.split("(")[0][3:]
                neighbour = x.split("(")[1][:-1]
                neighbour = neighbour.replace("/",",")
            else:
                borough = x[3:]
                neighbour = ""
        
            postals.append(postal)
            boroughs.append(borough)
            neighbours.append(neighbour)
dfCleaned = pd.DataFrame({"Postal Code": postals, "Borough": boroughs, "Neighbourhood":neighbours})
dfCleaned.shape


(103, 3)

## PART2: Getting the GeoLocations of the Neighbourhoods

Code provided by the project... enhanced to a function


In [3]:
#conda install -c conda-forge geocoder --yes
import time
timeout = time.time() +5
def getLatAndLongForPostal(postal):
    import geocoder # import geocoder
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal))
        #print('{}, Toronto, Ontario'.format(postal))
        lat_lng_coords = g.latlng
        if time.time() > timeout:
            break
    
    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    
    return lat_lng_coords

#coords = getLatAndLongForPostal('M5G')
#print (coords[0])

In [4]:
#longs = []
#lats = []

#for postal in dfCleaned["PostalCode"].values:
#    print(postal)
    #ll = getLatAndLongForPostal(postal)
    #longs.append(ll[0])
    #lats.append(ll[1])

#dfCleaned["Latitude"] = longs
#dfCleaned["Longitude"] = lats
#dfCleaned.head()

Unfortunately the code for geocoder did not work - so I had to use the csv instead


In [5]:
import requests
def downloadFile(url,fileName):
    fname = fileName
    url = url + "/" +fileName
    r = requests.get(url)
    open(fname , 'wb').write(r.content)
    print('Data downloaded!')
    return
#downloadFile("https://cocl.us/Geospatial_data",'Geospatial_Coordinates.csv')

coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()
#dfCleaned.head()

dfClustering = pd.merge(dfCleaned, coordinates, on="Postal Code")
dfClustering.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,,43.662301,-79.389494


Now to the Clustering

In [50]:
import folium
latitude = 43.651070
longitude = -79.347015
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfClustering['Latitude'], dfClustering['Longitude'], dfClustering['Borough'], dfClustering['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

First lets set up the hidden cell for the credentials 

In [7]:
{
    "tags": [
        "hide_input",
    ]
}
CLIENT_ID = '51XHGE1T5XDX0LJ1NS5YDWCEK2QT2NK3KF2BKSA5PLA1JPKB' # your Foursquare ID
CLIENT_SECRET = 'GQJ3FIKNEVYMZJH00NSS0KAEBPPEMCBJWHZEW4OLTSACG0S3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 51XHGE1T5XDX0LJ1NS5YDWCEK2QT2NK3KF2BKSA5PLA1JPKB
CLIENT_SECRET:GQJ3FIKNEVYMZJH00NSS0KAEBPPEMCBJWHZEW4OLTSACG0S3


## PART 3a: Analyzing the Neighborhoods

What is done here:
From all the Neighborhoods from all the Boroughs which have "Toronto in its Name"...find all venues within a radius of 500 around the borough center. Then - categorize them, count how many venues of each category are within each Neighborhood and finally sort them in decreasing order

The Clustering finds similar shaped neighborhoods with a fixed k (not such a good idea to be fair) 

Get all the Boroughs which start with "T"

In [12]:
toronto_data = dfClustering[dfClustering['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.shape

(39, 5)

Function from the previous lab to get the venues within a radius

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names=toronto_data['Borough'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
East YorkEast Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown TorontoStn A PO Boxes25 The Esplanade
Downtown Toronto
Downtown Toronto
Downtown Toronto
East TorontoBusiness reply mail Processing Centre969 Eastern


In [19]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,60,60,60,60,60,60
Downtown Toronto,164,164,164,164,164,164
Downtown TorontoStn A PO Boxes25 The Esplanade,10,10,10,10,10,10
East Toronto,34,34,34,34,34,34
East TorontoBusiness reply mail Processing Centre969 Eastern,10,10,10,10,10,10
East YorkEast Toronto,3,3,3,3,3,3
West Toronto,60,60,60,60,60,60


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 122 uniques categories.


Analyze the Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,...,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Central Toronto,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,...,0.050000,0.016667,0.000000,0.000000,0.000000,0.000000,0.033333,0.016667,0.000000,0.000000
1,Downtown Toronto,0.006098,0.006098,0.006098,0.006098,0.012195,0.006098,0.012195,0.006098,0.006098,...,0.006098,0.000000,0.018293,0.006098,0.006098,0.006098,0.006098,0.012195,0.006098,0.000000
2,Downtown TorontoStn A PO Boxes25 The Esplanade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000
3,East Toronto,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000
4,East TorontoBusiness reply mail Processing Cen...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,East YorkEast Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.016667,...,0.016667,0.000000,0.016667,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667


Bit more analyzing the data: What are the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
              venue  freq
0              Park  0.10
1       Coffee Shop  0.08
2              Café  0.05
3      Dessert Shop  0.05
4  Sushi Restaurant  0.05


----Downtown Toronto----
                venue  freq
0                Café  0.10
1         Coffee Shop  0.08
2          Restaurant  0.07
3                Park  0.04
4  Italian Restaurant  0.03


----Downtown TorontoStn A PO Boxes25 The Esplanade----
                           venue  freq
0                   Cocktail Bar   0.2
1                           Park   0.1
2              French Restaurant   0.1
3  Vegetarian / Vegan Restaurant   0.1
4                           Café   0.1


----East Toronto----
                venue  freq
0      Ice Cream Shop  0.12
1    Greek Restaurant  0.09
2                 Pub  0.09
3             Brewery  0.06
4  Italian Restaurant  0.06


----East TorontoBusiness reply mail Processing Centre969 Eastern----
                  venue  freq
0               Brewery   0.1
1         Ga

In [32]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Park,Coffee Shop,Dessert Shop,Café,Sushi Restaurant,Indian Restaurant,American Restaurant,Trail,Pub,Restaurant
1,Downtown Toronto,Café,Coffee Shop,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Pub,Tea Room
2,Downtown TorontoStn A PO Boxes25 The Esplanade,Cocktail Bar,Vegetarian / Vegan Restaurant,Park,Museum,French Restaurant,Thai Restaurant,Fountain,Restaurant,Café,Cuban Restaurant
3,East Toronto,Ice Cream Shop,Pub,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Yoga Studio,Health Food Store,Cosmetics Shop,Liquor Store
4,East TorontoBusiness reply mail Processing Cen...,Skate Park,Auto Workshop,Pizza Place,Burrito Place,Restaurant,Brewery,Farmers Market,Fast Food Restaurant,Comic Shop,Garden Center


## PART 3b: Real Clustering

In [75]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
# drop them first if they already exist
try: 
    neighborhoods_venues_sorted.drop(["Cluster Labels"], axis = 1,inplace = True)
except:
    print("no drop required")
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,2,Café,Coffee Shop,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Pub,Tea Room
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,2,Café,Coffee Shop,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Pub,Tea Room
2,Downtown Toronto,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,2,Café,Coffee Shop,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Pub,Tea Room
3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,2,Café,Coffee Shop,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Pub,Tea Room
4,Downtown Toronto,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,2,Café,Coffee Shop,Restaurant,Park,Italian Restaurant,Japanese Restaurant,Bakery,Gym,Pub,Tea Room


## Cluster visualization


In [76]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters